[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlos-adir/UnB-Courses/blob/main/vibracoes/multi-dofs-system.ipynb)

# 1 - Introduction

The given ODE we treat here is given by

$$
m\ddot{x} + c\dot{x} +kx = f
$$

With initial conditions $x(0) = x_0$ and $\dot{x}(0) = v_0$.

A helpful notation is

$$
\omega_n = \sqrt{\dfrac{k}{m}} \ \ \ \ \ \ \ \ \ \ \xi = \dfrac{c}{2\sqrt{km}} \ \ \ \ \ \ \ \ \ \ \mu = \sqrt{1-\xi^2}
$$

The solution for this ODE depends on the force $f$.

As it's a linear ODE we can always decompose the solution as

$$
x(t) = x_{h}(t) + x_{f}(t)
$$

Where

* $x_{h}$ represents the homogeneous response
    * $m\ddot{x}_{h} + c\dot{x}_{h} +kx_{h} = 0$
    * $x_{h}(0) = x_0$
    * $\dot{x}_{h}(0) = v_0$
* $x_f$ represents the forced response:
    * If $f=0$ then $x_{f}(t) = 0$
    * $x_{f}(0) = 0$ 
    * $\dot{x}_{f}(0) = 0$

Then, we are going to find the theoric solution for the homogeneous case and the forced solution.

# 2 - Theory

## 2.1 - Homogeneous solution

The homogeneous solution $x_h$ satisfies

$$m\ddot{x}_h + c\dot{x}_h +kx_h = 0$$

With initial conditions

$$
x(0) = x_0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \dot{x}(0) = v_0
$$

Then, we can divide the homogeneous solution using two base solutions

$$
x_h(t) = x_0  \cdot x_{p}(t) + v_0  \cdot x_{v}(t)
$$

With

$$
x_p(0) = 1 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \dot{x}(0) = 0
$$
$$
x_v(0) = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \dot{x}(0) = 1
$$

There are many ways to solve it. We are going to use Laplace transformation

$$
ms^2 \cdot X + cs \cdot X + kX = m \cdot x(0) + (ms+c) \cdot \dot{x}(0) = 0
$$
$$
X = \underbrace{\dfrac{m}{ms^2 + cs + k}}_{X_{p}} \cdot x(0) + \underbrace{\dfrac{ms+c}{ms^2 + cs + k}}_{X_{v}} \cdot \dot{x}
$$

The solution depends on the parameters $m$, $c$ and $k$, which we divide in three groups:

* Se $0\le \xi < 1$
> Temos a solução dada por
> 
> $$x_{p}(t) = \exp \left(-\xi \omega_n t\right) \cdot \left[\cos \mu \omega_n t + \dfrac{\xi}{\mu} \cdot \sin \mu \omega_n t\right]$$
> $$x_{v}(t) = \dfrac{1}{\mu \omega_{n}}\exp \left(-\xi \omega_n t\right) \cdot \sin \mu \omega_n t$$
> Cujas derivadas são
> 
> $$\dot{x}_p(t) = - \omega_n \cdot \exp \left(-\xi \omega_n t\right) \cdot \sin \mu \omega_n t$$
> $$\dot{x}_v(t) = \exp \left(-\xi \omega_n t\right) \cdot \left[\cos \mu \omega_n t - \dfrac{\xi}{\mu} \cdot \sin \mu \omega_n t\right] $$
> 
* Se $\xi = 1$
> Temos a solução dada por
> 
> $$x_{p}(t) = \left(1+\omega_n t\right) \exp \left(- \omega_n t\right) $$
> $$x_{v}(t) = t \cdot \exp \left(-\xi \omega_n t\right)$$
>
> Cujas derivadas são
>
> $$\dot{x}_p(t) = - \omega_n \cdot \exp \left(-\xi \omega_n t\right) \cdot \sin \mu \omega_n t$$
> $$\dot{x}_v(t) = \exp \left(-\xi \omega_n t\right) \cdot \left[\cos \mu \omega_n t - \dfrac{\xi}{\mu} \cdot \sin \mu \omega_n t\right] $$
> 
* Se $\xi > 1$
> 
> Para esse específico caso temos que
> $$\mu = \sqrt{\xi^2 - 1}\ \ \ \ \ \ \ \ \ \ \ \ \alpha = \xi - \mu\ \ \ \ \ \ \ \ \ \ \ \ \beta = \xi + \mu$$
> 
> Disto, temos a solução dada por
> 
> $$x_p(t) = \dfrac{1}{2\mu}\left[\beta  \cdot \exp \left(-\alpha \omega_n t\right) - \alpha \cdot \exp \left(-\beta \omega_n t\right)\right]$$
> $$x_v(t) = \dfrac{1}{2\mu\omega_n} \cdot \left[ \exp \left(-\alpha \omega_n t\right) - \exp \left(-\beta \omega_n t\right)\right]$$
>
> Cujas derivadas são
>
> $$\dot{x}_p(t) = \dfrac{-\alpha\beta \omega_n}{2\mu}  \left[ \exp \left(-\alpha \omega_n t\right) -  \exp \left(-\beta \omega_n t\right)\right]$$
> $$\dot{x}_v(t) = \dfrac{1}{2\mu} \left[ \beta \exp \left(-\beta \omega_n t\right) - \alpha \cdot \exp \left(-\alpha \omega_n t\right) \right] $$

This OD